In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 217975


,eng,fra,cc
188457,All the students protested against the war.,Tous les étudiants ont protesté contre la guerre.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
56518,It's not a fair fight.,Ce n'est pas un combat loyal.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
14177,Did you read it?,Est-ce que vous l'avez lu ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
202445,We spent a lot of time looking for a parking lot.,Nous avons passé beaucoup de temps à chercher ...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
105109,I grew these carrots myself.,J'ai fait moi-même pousser ces carottes.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [3]:
lines = lines[['eng', 'fra']][:8800] # 1.3만개 샘플 사용
lines.sample(5)

,eng,fra
3748,Let's hurry.,Dépêchons-nous.
2671,We're even.,Nous sommes quittes.
1967,I enjoy it.,J'apprécie cela.
3142,I can't see.,Je ne parviens pas à voir.
262,Get out.,Sortez !


In [4]:
# 시작 토큰과 종료 토큰 추가
sos_token = 'sos'
eos_token = 'eos'
lines.fra = lines.fra.apply(lambda x : 'sos ' + x + ' eos')
print('전체 샘플의 수 :',len(lines))
lines.sample(5)

전체 샘플의 수 : 8800


,eng,fra
5823,Let me do it.,sos Laisse-moi faire. eos
8041,I'll take one.,sos J'en prendrai une. eos
3296,I must hide.,sos Il me faut me cacher. eos
4229,Was it good?,sos Était-ce bon ? eos
406,Shut up!,sos Ferme-la ! eos


In [5]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

lines.eng = lines.eng.apply(lambda x : text_to_word_sequence(x))
lines.fra = lines.fra.apply(lambda x : text_to_word_sequence(x))
# lines.eng = lines.eng.apply(lambda x : x.lower())
# lines.fra = lines.fra.apply(lambda x : x.lower())

lines.sample(5)

,eng,fra
6208,"[they're, free]","[sos, ils, sont, libres, eos]"
4389,"[where's, tom]","[sos, où, est, tom, eos]"
2374,"[look, at, it]","[sos, regarde, le, eos]"
7862,"[i, see, the, boy]","[sos, je, vois, le, garçon, eos]"
5473,"[i'm, not, dead]","[sos, je, ne, suis, pas, mort, eos]"


In [6]:
eng_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
eng_tokenizer.fit_on_texts(lines.eng)               # 50000개의 행을 가진 eng의 각 행에 토큰화를 수행
input_text = eng_tokenizer.texts_to_sequences(lines.eng)    # 단어를 숫자값 인덱스로 변환하여 저장
input_text[:30]

[[9],
 [9],
 [9],
 [9],
 [366],
 [366],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [86],
 [46],
 [760],
 [760],
 [760],
 [761],
 [761],
 [761],
 [367]]

In [7]:
fra_tokenizer = Tokenizer()   # 문자 단위로 Tokenizer를 생성합니다. 
fra_tokenizer.fit_on_texts(lines.fra)                 # 50000개의 행을 가진 fra의 각 행에 토큰화를 수행
target_text = fra_tokenizer.texts_to_sequences(lines.fra)     # 단어를 숫자값 인덱스로 변환하여 저장
target_text[:30]

[[1, 37, 2],
 [1, 210, 2],
 [1, 22, 246, 2],
 [1, 197, 2],
 [1, 413, 2],
 [1, 413, 2],
 [1, 1174, 2],
 [1, 1175, 2],
 [1, 81, 259, 470, 16, 259, 859, 2],
 [1, 471, 2],
 [1, 860, 2],
 [1, 472, 2],
 [1, 861, 2],
 [1, 1176, 2],
 [1, 1174, 2],
 [1, 1175, 2],
 [1, 81, 259, 470, 16, 259, 859, 2],
 [1, 471, 2],
 [1, 860, 2],
 [1, 472, 2],
 [1, 861, 2],
 [1, 1176, 2],
 [1, 30, 2],
 [1, 20, 1928, 2],
 [1, 1929, 2],
 [1, 1930, 2],
 [1, 16, 1931, 2],
 [1, 665, 862, 2],
 [1, 1932, 1177, 2],
 [1, 71, 414, 2]]

In [8]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 1844
프랑스어 단어장의 크기 : 3984


In [9]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 4
프랑스어 시퀀스의 최대 길이 12


In [10]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 8800
영어 단어장의 크기 : 1844
프랑스어 단어장의 크기 : 3984
영어 시퀀스의 최대 길이 4
프랑스어 시퀀스의 최대 길이 12


In [11]:
encoder_input = input_text
# 종료 토큰 제거
decoder_input = [[ char for char in line if char != fra_tokenizer.word_index[eos_token] ] for line in target_text] 
# 시작 토큰 제거
decoder_target = [[ char for char in line if char != fra_tokenizer.word_index[sos_token] ] for line in target_text]

In [12]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 37], [1, 210], [1, 22, 246]]
[[37, 2], [210, 2], [22, 246, 2]]


In [13]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (8800, 4)
프랑스어 입력데이터의 크기(shape) : (8800, 12)
프랑스어 출력데이터의 크기(shape) : (8800, 12)


In [14]:
print(encoder_input[0])

[9 0 0 0]


In [15]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (8800, 4, 1844)
프랑스어 입력데이터의 크기(shape) : (8800, 12, 3984)
프랑스어 출력데이터의 크기(shape) : (8800, 12, 3984)


In [16]:
n_of_val = 800

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (8000, 4, 1844)
프랑스어 학습 입력데이터의 크기(shape) : (8000, 12, 3984)
프랑스어 학습 출력데이터의 크기(shape) : (8000, 12, 3984)


In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

print('⏳')

⏳


In [18]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
encoder_states = [state_h, state_c]

In [19]:
# 입력 텐서 생성.
decoder_inputs = Input(shape=(None, fra_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
decoder_lstm = LSTM(units = 256, return_sequences = True, return_state=True)
# decoder_outputs는 모든 time step의 hidden state
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state = encoder_states)

In [20]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [21]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 1844)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 3984)] 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 2151424     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  4342784     input_2[0][0]                    
                                                                 lstm[0][1]                   

In [22]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Epoch 1/50
63/63 [==============================] - 8s 59ms/step - loss: 2.5610 - val_loss: 2.1129
Epoch 2/50
63/63 [==============================] - 2s 36ms/step - loss: 1.8031 - val_loss: 1.9854
Epoch 3/50
63/63 [==============================] - 2s 36ms/step - loss: 1.6921 - val_loss: 1.9276
Epoch 4/50
63/63 [==============================] - 2s 36ms/step - loss: 1.5970 - val_loss: 1.8463
Epoch 5/50
63/63 [==============================] - 2s 36ms/step - loss: 1.5056 - val_loss: 1.7772
Epoch 6/50
63/63 [==============================] - 2s 36ms/step - loss: 1.4205 - val_loss: 1.7335
Epoch 7/50
63/63 [==============================] - 2s 36ms/step - loss: 1.3354 - val_loss: 1.6895
Epoch 8/50
63/63 [==============================] - 2s 36ms/step - loss: 1.2569 - val_loss: 1.6168
Epoch 9/50
63/63 [==============================] - 2s 36ms/step - loss: 1.1858 - val_loss: 1.5919
Epoch 10/50
63/63 [==============================] - 2s 35ms/step - loss: 1.1267 - val_loss: 1.5456
Epoch 11/

In [23]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 1844)]      0         
_________________________________________________________________
lstm (LSTM)                  [(None, 256), (None, 256) 2151424   
Total params: 2,151,424
Trainable params: 2,151,424
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [25]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 3984)] 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  4342784     input_2[0][0]                    
                                                                 input_3[0][0]              

In [26]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [27]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['sos']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == 'eos' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [28]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-3]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: ['go']
정답 문장: ['bouge']
번역기가 번역한 문장: va
-----------------------------------
입력 문장: ['hello']
정답 문장: ['bonjour']
번역기가 번역한 문장: bonjour
-----------------------------------
입력 문장: ['got', 'it']
정답 문장: ['compris']
번역기가 번역한 문장: ça
-----------------------------------
입력 문장: ['goodbye']
정답 문장: ['au', 'revoir']
번역기가 번역한 문장: ciao
-----------------------------------
입력 문장: ['hands', 'off']
정답 문장: ['pas', 'touche\u202f']
번역기가 번역한 문장: pastouche 
